Ispired by this notebook.

https://www.kaggle.com/mrkmakr/covid-ae-pretrain-gnn-attn-cnn

Pytorch Implementation.

Model: Convolution + Transfomer + GRU.

Training: Bert-Like Pretraining. Then Fine Tuning.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/openwaccineaugdata/aug_df2.json
/kaggle/input/stanford-covid-vaccine/test.json
/kaggle/input/stanford-covid-vaccine/train.json
/kaggle/input/stanford-covid-vaccine/sample_submission.csv
/kaggle/input/stanford-covid-vaccine/bpps/id_7adcb8c5d.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_1874dad2f.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_2e5c35220.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_dfcd87ac5.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_4c50e9deb.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_a7e61b2bf.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_941abeff3.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_980bf19cd.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_edbeb2164.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_4bd0225a2.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_43cbbe539.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_5b2e84c62.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_a82338fe2.npy
/kaggle/input/stanford-covid-vacc

In [2]:
from collections import OrderedDict
from fastprogress import progress_bar
from pathlib import Path
from sklearn.model_selection import train_test_split, ShuffleSplit
from torch import nn
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

import functools
import os
import pandas as pd
import random
import shutil
import torch
import torch.nn.functional as F


target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
input_cols = ['sequence', 'structure', 'predicted_loop_type']
error_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_Mg_50C', 'deg_error_pH10', 'deg_error_50C']

token_dicts = {
    "sequence": {x: i for i, x in enumerate("ACGU")},
    "structure": {x: i for i, x in enumerate('().')},
    "predicted_loop_type": {x: i for i, x in enumerate("BEHIMSX")}
}


def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

### Loader


In [3]:
from sklearn.model_selection import train_test_split, ShuffleSplit
from torch import nn
from torch.utils.data import Dataset

import functools


BASE_PATH = "/kaggle/input/stanford-covid-vaccine"
MODEL_SAVE_PATH = "/kaggle/model"


def preprocess_inputs(df, cols):
    return np.concatenate([preprocess_feature_col(df, col) for col in cols], axis=2)


def preprocess_feature_col(df, col):
    dic = token_dicts[col]
    dic_len = len(dic)
    seq_length = len(df[col][0])
    ident = np.identity(dic_len)
    # convert to one hot
    arr = np.array(
        df[[col]].applymap(lambda seq: [ident[dic[x]] for x in seq]).values.tolist()
    ).squeeze(1)
    # shape: data_size x seq_length x dic_length
    assert arr.shape == (len(df), seq_length, dic_len)
    return arr


def preprocess(base_data, is_test=False):
    inputs = preprocess_inputs(base_data, input_cols)
    if is_test:
        labels = None
    else:
        labels = np.array(base_data[target_cols].values.tolist()).transpose((0, 2, 1))
        assert labels.shape[2] == len(target_cols)
    assert inputs.shape[2] == 14
    return inputs, labels


def get_bpp_feature(bpp):
    bpp_nb_mean = 0.077522  # mean of bpps_nb across all training data
    bpp_nb_std = 0.08914  # std of bpps_nb across all training data
    bpp_max = bpp.max(-1)[0]
    bpp_sum = bpp.sum(-1)
    bpp_nb = torch.true_divide((bpp > 0).sum(dim=1), bpp.shape[1])
    bpp_nb = torch.true_divide(bpp_nb - bpp_nb_mean, bpp_nb_std)
    return [bpp_max.unsqueeze(2), bpp_sum.unsqueeze(2), bpp_nb.unsqueeze(2)]


@functools.lru_cache(5000)
def load_from_id(id_):
    path = Path(BASE_PATH) / f"bpps/{id_}.npy"
    data = np.load(str(path))
    return data


def get_distance_matrix(leng):
    idx = np.arange(leng)
    Ds = []
    for i in range(len(idx)):
        d = np.abs(idx[i] - idx)
        Ds.append(d)

    Ds = np.array(Ds) + 1
    Ds = 1 / Ds
    Ds = Ds[None, :, :]
    Ds = np.repeat(Ds, 1, axis=0)

    Dss = []
    for i in [1, 2, 4]:
        Dss.append(Ds ** i)
    Ds = np.stack(Dss, axis=3)
    print(Ds.shape)
    return Ds


def get_structure_adj(df):
    Ss = []
    for i in range(len(df)):
        seq_length = df["seq_length"].iloc[i]
        structure = df["structure"].iloc[i]
        sequence = df["sequence"].iloc[i]

        cue = []
        a_structures = OrderedDict([
            (("A", "U"), np.zeros([seq_length, seq_length])),
            (("C", "G"), np.zeros([seq_length, seq_length])),
            (("U", "G"), np.zeros([seq_length, seq_length])),
            (("U", "A"), np.zeros([seq_length, seq_length])),
            (("G", "C"), np.zeros([seq_length, seq_length])),
            (("G", "U"), np.zeros([seq_length, seq_length])),
        ])
        for j in range(seq_length):
            if structure[j] == "(":
                cue.append(j)
            elif structure[j] == ")":
                start = cue.pop()
                a_structures[(sequence[start], sequence[j])][start, j] = 1
                a_structures[(sequence[j], sequence[start])][j, start] = 1

        a_strc = np.stack([a for a in a_structures.values()], axis=2)
        a_strc = np.sum(a_strc, axis=2, keepdims=True)
        Ss.append(a_strc)

    Ss = np.array(Ss)
    return Ss


def create_loader(df, batch_size=1, is_test=False):
    features, labels = preprocess(df, is_test)
    features_tensor = torch.from_numpy(features)
    if labels is not None:
        labels_tensor = torch.from_numpy(labels)
        dataset = VacDataset(features_tensor, df, labels_tensor)
        loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True, drop_last=False)
    else:
        dataset = VacDataset(features_tensor, df, None)
        loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=False, drop_last=False)
    return loader


class VacDataset(Dataset):
    def __init__(self, features, df, labels=None):
        self.features = features
        self.labels = labels
        self.test = labels is None
        self.ids = df["id"]
        self.score = None
        self.structure_adj = get_structure_adj(df)
        self.distance_matrix = get_distance_matrix(self.structure_adj.shape[1])
        if "score" in df.columns:
            self.score = df["score"]
        else:
            df["score"] = 1.0
            self.score = df["score"]
        self.signal_to_noise = None
        if not self.test:
            self.signal_to_noise = df["signal_to_noise"]
            assert self.features.shape[0] == self.labels.shape[0]
        else:
            assert self.ids is not None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        bpp = torch.from_numpy(load_from_id(self.ids[index]).copy()).float()
        adj = self.structure_adj[index]
        distance = self.distance_matrix[0]
        bpp = np.concatenate([bpp[:, :, None], adj, distance], axis=2)
        if self.test:
            return dict(sequence=self.features[index].float(), bpp=bpp, ids=self.ids[index])
        else:
            return dict(sequence=self.features[index].float(), bpp=bpp,
                        label=self.labels[index], ids=self.ids[index],
                        signal_to_noise=self.signal_to_noise[index],
                        score=self.score[index])


### Model

In [4]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer

import math


class Conv1dStack(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size=3, padding=1, dilation=1):
        super(Conv1dStack, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm1d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )
        self.res = nn.Sequential(
            nn.Conv1d(out_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm1d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )

    def forward(self, x):
        x = self.conv(x)
        h = self.res(x)
        return x + h


class Conv2dStack(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size=3, padding=1, dilation=1):
        super(Conv2dStack, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )
        self.res = nn.Sequential(
            nn.Conv2d(out_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )

    def forward(self, x):
        x = self.conv(x)
        h = self.res(x)
        return x + h


class SeqEncoder(nn.Module):
    def __init__(self, in_dim: int):
        super(SeqEncoder, self).__init__()
        self.conv0 = Conv1dStack(in_dim, 128, 3, padding=1)
        self.conv1 = Conv1dStack(128, 64, 6, padding=5, dilation=2)
        self.conv2 = Conv1dStack(64, 32, 15, padding=7, dilation=1)
        self.conv3 = Conv1dStack(32, 32, 30, padding=29, dilation=2)

    def forward(self, x):
        x1 = self.conv0(x)
        x2 = self.conv1(x1)
        x3 = self.conv2(x2)
        x4 = self.conv3(x3)
        x = torch.cat([x1, x2, x3, x4], dim=1)
        # x = x.permute(0, 2, 1).contiguous()
        # BATCH x 256 x seq_length
        return x


class BppAttn(nn.Module):
    def __init__(self, in_channel: int, out_channel: int):
        super(BppAttn, self).__init__()
        self.conv0 = Conv1dStack(in_channel, out_channel, 3, padding=1)
        self.bpp_conv = Conv2dStack(5, out_channel)

    def forward(self, x, bpp):
        x = self.conv0(x)
        bpp = self.bpp_conv(bpp)
        # BATCH x C x SEQ x SEQ
        # BATCH x C x SEQ
        x = torch.matmul(bpp, x.unsqueeze(-1))
        return x.squeeze(-1)


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerWrapper(nn.Module):
    def __init__(self, dmodel=256, nhead=8, num_layers=2):
        super(TransformerWrapper, self).__init__()
        self.pos_encoder = PositionalEncoding(256)
        encoder_layer = TransformerEncoderLayer(d_model=dmodel, nhead=nhead)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers)
        self.pos_emb = PositionalEncoding(dmodel)

    def flatten_parameters(self):
        pass

    def forward(self, x):
        x = x.permute((1, 0, 2)).contiguous()
        x = self.pos_emb(x)
        x = self.transformer_encoder(x)
        x = x.permute((1, 0, 2)).contiguous()
        return x, None


class RnnLayers(nn.Module):
    def __init__(self, dmodel, dropout=0.3, transformer_layers: int = 2):
        super(RnnLayers, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.rnn0 = TransformerWrapper(dmodel, nhead=8, num_layers=transformer_layers)
        self.rnn1 = nn.LSTM(dmodel, dmodel // 2, batch_first=True, num_layers=1, bidirectional=True)
        self.rnn2 = nn.GRU(dmodel, dmodel // 2, batch_first=True, num_layers=1, bidirectional=True)

    def forward(self, x):
        self.rnn0.flatten_parameters()
        x, _ = self.rnn0(x)
        if self.rnn1 is not None:
            self.rnn1.flatten_parameters()
            x = self.dropout(x)
            x, _ = self.rnn1(x)
        if self.rnn2 is not None:
            self.rnn2.flatten_parameters()
            x = self.dropout(x)
            x, _ = self.rnn2(x)
        return x

    
class BaseAttnModel(nn.Module):
    def __init__(self, transformer_layers: int = 2):
        super(BaseAttnModel, self).__init__()
        self.linear0 = nn.Linear(14 + 3, 1)
        self.seq_encoder_x = SeqEncoder(18)
        self.attn = BppAttn(256, 128)
        self.seq_encoder_bpp = SeqEncoder(128)
        self.seq = RnnLayers(256 * 2, dropout=0.3,
                             transformer_layers=transformer_layers)

    def forward(self, x, bpp):
        bpp_features = get_bpp_feature(bpp[:, :, :, 0].float())
        x = torch.cat([x] + bpp_features, dim=-1)
        learned = self.linear0(x)
        x = torch.cat([x, learned], dim=-1)
        x = x.permute(0, 2, 1).contiguous().float()
        # BATCH x 18 x seq_len
        bpp = bpp.permute([0, 3, 1, 2]).contiguous().float()
        # BATCH x 5 x seq_len x seq_len
        x = self.seq_encoder_x(x)
        # BATCH x 256 x seq_len
        bpp = self.attn(x, bpp)
        bpp = self.seq_encoder_bpp(bpp)
        # BATCH x 256 x seq_len
        x = x.permute(0, 2, 1).contiguous()
        # BATCH x seq_len x 256
        bpp = bpp.permute(0, 2, 1).contiguous()
        # BATCH x seq_len x 256
        x = torch.cat([x, bpp], dim=2)
        # BATCH x seq_len x 512
        x = self.seq(x)
        return x


class AEModel(nn.Module):
    def __init__(self, transformer_layers: int = 2):
        super(AEModel, self).__init__()
        self.seq = BaseAttnModel(transformer_layers=transformer_layers)
        self.linear = nn.Sequential(
            nn.Linear(256 * 2, 14),
            nn.Sigmoid(),
        )

    def forward(self, x, bpp):
        x = self.seq(x, bpp)
        x = F.dropout(x, p=0.3)
        x = self.linear(x)
        return x


class FromAeModel(nn.Module):
    def __init__(self, seq, pred_len=68, dmodel: int = 256):
        super(FromAeModel, self).__init__()
        self.seq = seq
        self.pred_len = pred_len
        self.linear = nn.Sequential(
            nn.Linear(dmodel * 2, len(target_cols)),
        )

    def forward(self, x, bpp):
        x = self.seq(x, bpp)
        x = self.linear(x)
        x = x[:, :self.pred_len]
        return x


In [5]:
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
base_train_data.head()

device = torch.device('cuda')
BATCH_SIZE = 64
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
base_test_data = pd.read_json(str(Path(BASE_PATH) / 'test.json'), lines=True)
public_df = base_test_data.query("seq_length == 107").copy()
private_df = base_test_data.query("seq_length == 130").copy()
print(f"public_df: {public_df.shape}")
print(f"private_df: {private_df.shape}")
public_df = public_df.reset_index()
private_df = private_df.reset_index()

features, _ = preprocess(base_train_data, True)
features_tensor = torch.from_numpy(features)
dataset0 = VacDataset(features_tensor, base_train_data, None)
features, _ = preprocess(public_df, True)
features_tensor = torch.from_numpy(features)
dataset1 = VacDataset(features_tensor, public_df, None)
features, _ = preprocess(private_df, True)
features_tensor = torch.from_numpy(features)
dataset2 = VacDataset(features_tensor, private_df, None)

loader0 = torch.utils.data.DataLoader(dataset0, BATCH_SIZE, shuffle=False, drop_last=False)
loader1 = torch.utils.data.DataLoader(dataset1, BATCH_SIZE, shuffle=False, drop_last=False)
loader2 = torch.utils.data.DataLoader(dataset2, BATCH_SIZE, shuffle=False, drop_last=False)

public_df: (629, 7)
private_df: (3005, 7)
(1, 107, 107, 3)
(1, 107, 107, 3)
(1, 130, 130, 3)


### Pretrain

In [6]:
def learn_from_batch_ae(model, data, device):
    seq = data["sequence"].clone()
    seq[:, :, :14] = F.dropout2d(seq[:, :, :14], p=0.3)
    target = data["sequence"][:, :, :14]
    out = model(seq.to(device), data["bpp"].to(device))
    loss = F.binary_cross_entropy(out, target.to(device))
    return loss


def train_ae(model, train_data, optimizer, lr_scheduler, epochs=10, device="cpu",
             start_epoch: int = 0, start_it: int = 0, log_path: str = "./logs"):
    print(f"device: {device}")
    losses = []
    it = start_it
    model_save_path = Path(MODEL_SAVE_PATH)
    start_epoch = start_epoch
    end_epoch = start_epoch + epochs
    min_loss = 10.0
    min_loss_epoch = 0
    if not model_save_path.exists():
        model_save_path.mkdir(parents=True)
    for epoch in progress_bar(range(start_epoch, end_epoch)):
        print(f"epoch: {epoch}")
        model.train()
        for i, data in enumerate(train_data):
            optimizer.zero_grad()
            loss = learn_from_batch_ae(model, data, device)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            if lr_scheduler:
                lr_scheduler.step()
            loss_v = loss.item()
            losses.append(loss_v)
            it += 1
        loss_m = np.mean(losses)
        if loss_m < min_loss:
            min_loss_epoch = epoch
            min_loss = loss_m
        print(f'epoch: {epoch} loss: {loss_m}')
        losses = []
        torch.save(optimizer.state_dict(), str(model_save_path / "optimizer.pt"))
        torch.save(model.state_dict(), str(model_save_path / f"model-{epoch}.pt"))
    return dict(end_epoch=end_epoch, it=it, min_loss_epoch=min_loss_epoch)


In [7]:
import shutil


set_seed(123)
shutil.rmtree("./model", True)
shutil.rmtree("./logs", True)
save_path = Path("./model_prediction")
if not save_path.exists():
    save_path.mkdir(parents=True)

lr_scheduler = None
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AEModel()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
res = dict(end_epoch=0, it=0, min_loss_epoch=0)
epochs = [5, 5, 5, 5]
for e in epochs:
    res = train_ae(model, loader0, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])
    res = train_ae(model, loader1, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])
    res = train_ae(model, loader2, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])

epoch = res["min_loss_epoch"]
shutil.copyfile(str(Path(MODEL_SAVE_PATH) / f"model-{epoch}.pt"), "ae-model.pt")

device: cuda


epoch: 0
epoch: 0 loss: 0.32869393104001093
epoch: 1
epoch: 1 loss: 0.1732125850884538
epoch: 2
epoch: 2 loss: 0.13422635079998718
epoch: 3
epoch: 3 loss: 0.10131922070133059
epoch: 4
epoch: 4 loss: 0.07272145544227801
device: cuda


epoch: 5
epoch: 5 loss: 0.046084293723106386
epoch: 6
epoch: 6 loss: 0.03665613271296024
epoch: 7
epoch: 7 loss: 0.03188155796378851
epoch: 8
epoch: 8 loss: 0.02604033052921295
epoch: 9
epoch: 9 loss: 0.02375647071748972
device: cuda


epoch: 10
epoch: 10 loss: 0.026184827287463432
epoch: 11
epoch: 11 loss: 0.020211882057025076
epoch: 12
epoch: 12 loss: 0.01779099320002059
epoch: 13
epoch: 13 loss: 0.016356333277802518
epoch: 14
epoch: 14 loss: 0.01535436885550301
device: cuda


epoch: 15
epoch: 15 loss: 0.011690631263742321
epoch: 16
epoch: 16 loss: 0.009787665140864096
epoch: 17
epoch: 17 loss: 0.009324878490971108
epoch: 18
epoch: 18 loss: 0.008570758307254627
epoch: 19
epoch: 19 loss: 0.00801830681187934
device: cuda


...

epoch: 40
epoch: 40 loss: 0.01586846937920819
epoch: 41
epoch: 41 loss: 0.014356984340764106
epoch: 42
epoch: 42 loss: 0.014078114280833843
epoch: 43
epoch: 43 loss: 0.01336363286889614
epoch: 44
epoch: 44 loss: 0.012917424730480985
device: cuda


epoch: 45
epoch: 45 loss: 0.008213160559535027
epoch: 46
epoch: 46 loss: 0.007687914822446673
epoch: 47
epoch: 47 loss: 0.007608285014468588
epoch: 48
epoch: 48 loss: 0.007619376208535151
epoch: 49
epoch: 49 loss: 0.007356812773076327
device: cuda


epoch: 50
epoch: 50 loss: 0.006053892336785794
epoch: 51
epoch: 51 loss: 0.006183118699118495
epoch: 52
epoch: 52 loss: 0.005673004221171141
epoch: 53
epoch: 53 loss: 0.005450429720804095
epoch: 54
epoch: 54 loss: 0.0056069723796099424
device: cuda


epoch: 55
epoch: 55 loss: 0.012586911129666136
epoch: 56
epoch: 56 loss: 0.011855349380602228
epoch: 57
epoch: 57 loss: 0.011501792600338763
epoch: 58
epoch: 58 loss: 0.011094992088669159
epoch: 59
epoch: 59 loss: 0.011110003879095645


'ae-model.pt'

### Training

In [8]:
def MCRMSE(y_true, y_pred):
    colwise_mse = torch.mean(torch.square(y_true - y_pred), dim=1)
    return torch.mean(torch.sqrt(colwise_mse), dim=1)


def sn_mcrmse_loss(predict, target, signal_to_noise):
    loss = MCRMSE(target, predict)
    weight = 0.5 * torch.log(signal_to_noise + 1.01)
    loss = (loss * weight).mean()
    return loss


def learn_from_batch(model, data, optimizer, lr_scheduler, device):
    optimizer.zero_grad()
    out = model(data["sequence"].to(device), data["bpp"].to(device))
    signal_to_noise = data["signal_to_noise"] * data["score"]
    loss = sn_mcrmse_loss(out, data["label"].to(device), signal_to_noise.to(device))
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()
    if lr_scheduler:
        lr_scheduler.step()
    return out, loss


def evaluate(model, valid_data, device):
    model.eval()
    loss_list = []
    mcrmse = []
    for i, data in enumerate(valid_data):
        with torch.no_grad():
            y = model(data["sequence"].to(device), data["bpp"].to(device))
            mcrmse_ = MCRMSE(data["label"].to(device), y)[data["signal_to_noise"] > 1]
            mcrmse.append(mcrmse_.mean().item())
            loss = sn_mcrmse_loss(y, data["label"].to(device), data["signal_to_noise"].to(device))
            loss_list.append(loss.item())
    model.train()
    return dict(loss=np.mean(loss_list), mcmse=np.mean(mcrmse))


def train(model, train_data, valid_data, optimizer, lr_scheduler, epochs=10, device="cpu",
          start_epoch: int = 0, log_path: str = "./logs"):
    print(f"device: {device}")
    losses = []
    writer = SummaryWriter(log_path)
    it = 0
    model_save_path = Path(MODEL_SAVE_PATH)
    start_epoch = start_epoch
    end_epoch = start_epoch + epochs
    if not model_save_path.exists():
        model_save_path.mkdir(parents=True)
    min_eval_loss = 10.0
    min_eval_epoch = None
    for epoch in progress_bar(range(start_epoch, end_epoch)):
        print(f"epoch: {epoch}")
        model.train()
        for i, data in enumerate(train_data):
            _, loss = learn_from_batch(model, data, optimizer, lr_scheduler, device)
            loss_v = loss.item()
            writer.add_scalar('loss', loss_v, it)
            losses.append(loss_v)
            it += 1
        print(f'epoch: {epoch} loss: {np.mean(losses)}')
        losses = []

        eval_result = evaluate(model, valid_data, device)
        eval_loss = eval_result["loss"]
        if eval_loss <= min_eval_loss:
            min_eval_epoch = epoch
            min_eval_loss = eval_loss

        print(f"eval loss: {eval_loss} {eval_result['mcmse']}")
        writer.add_scalar(f"evaluate/loss", eval_loss, epoch)
        writer.add_scalar(f"evaluate/mcmse", eval_result["mcmse"], epoch)
        model.train()
        torch.save(optimizer.state_dict(), str(model_save_path / "optimizer.pt"))
        torch.save(model.state_dict(), str(model_save_path / f"model-{epoch}.pt"))
    print(f'min eval loss: {min_eval_loss} epoch {min_eval_epoch}')
    return min_eval_epoch


In [9]:
device = torch.device('cuda')
BATCH_SIZE = 64
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
samples = base_train_data
save_path = Path("./model_prediction")
if not save_path.exists():
    save_path.mkdir(parents=True)
shutil.rmtree("./model", True)
shutil.rmtree("./logs", True)
split = ShuffleSplit(n_splits=5, test_size=.1)
ids = samples.reset_index()["id"]
set_seed(124)
for fold, (train_index, test_index) in enumerate(split.split(samples)):
    print(f"fold: {fold}")
    train_df = samples.loc[train_index].reset_index()
    val_df = samples.loc[test_index].reset_index()
    train_loader = create_loader(train_df, BATCH_SIZE)
    valid_loader = create_loader(val_df, BATCH_SIZE)
    print(train_df.shape, val_df.shape)
    ae_model = AEModel()
    state_dict = torch.load("./ae-model.pt")
    ae_model.load_state_dict(state_dict)
    del state_dict
    model = FromAeModel(ae_model.seq)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    lr_scheduler = None
    epoch = train(model, train_loader, valid_loader, optimizer, lr_scheduler, 200, device=device,
                  log_path=f"logs/{fold}")
    shutil.copyfile(str(Path(MODEL_SAVE_PATH) / f"./model-{epoch}.pt"), f"model_prediction/model-{fold}.pt")
    del model

fold: 0
(1, 107, 107, 3)
(1, 107, 107, 3)
(2160, 21) (240, 21)
device: cuda


epoch: 0
epoch: 0 loss: 0.30934015436833917
eval loss: 0.2508997617694583 0.3282543899614309
epoch: 1
epoch: 1 loss: 0.2476099410579828
eval loss: 0.23337266998381026 0.30648029106895425
epoch: 2
epoch: 2 loss: 0.23316761807885045
eval loss: 0.2228317124173043 0.2908900974060071
epoch: 3
epoch: 3 loss: 0.22396023264046389
eval loss: 0.21687749565749395 0.2850249169795995
epoch: 4
epoch: 4 loss: 0.21663490634241542
eval loss: 0.2118985770544521 0.27772005366803215
epoch: 5
epoch: 5 loss: 0.21045051532343004
eval loss: 0.21268776765474015 0.27977675986057987
epoch: 6
epoch: 6 loss: 0.20551812826577995
eval loss: 0.20169907032615175 0.2638220160567483
epoch: 7
epoch: 7 loss: 0.20020086067443835
eval loss: 0.19342048233571613 0.2545356928525894
epoch: 8
epoch: 8 loss: 0.19605179819745666
eval loss: 0.19164833061543474 0.25193553740342206
epoch: 9
epoch: 9 loss: 0.19269512084136983
eval loss: 0.18696400020196963 0.24534158470987075
epoch: 10
epoch: 10 loss: 0.18878044318964257
eval loss: 0.

epoch: 0
epoch: 0 loss: 0.3181987736482989
eval loss: 0.269012711874039 0.34476528549723845
epoch: 1
epoch: 1 loss: 0.25142742173912735
eval loss: 0.2502987958584305 0.32175619466895705
epoch: 2
epoch: 2 loss: 0.23724350052074897
eval loss: 0.2366493922613413 0.30300353778159883
epoch: 3
epoch: 3 loss: 0.22876732599606867
eval loss: 0.23081720523103616 0.29675948936631014
epoch: 4
epoch: 4 loss: 0.22235602177546504
eval loss: 0.2268556522734344 0.290205301724517
epoch: 5
epoch: 5 loss: 0.21485286223850888
eval loss: 0.21844033779238053 0.2810966452817575
epoch: 6
epoch: 6 loss: 0.21035657798096669
eval loss: 0.21670661149395126 0.2786513677949892
epoch: 7
epoch: 7 loss: 0.20612980609174195
eval loss: 0.21014767178400767 0.27030057744550323
epoch: 8
epoch: 8 loss: 0.20026288136918355
eval loss: 0.2076115725443952 0.2667314803809023
epoch: 9
epoch: 9 loss: 0.1964911299651603
eval loss: 0.20514962913518547 0.26438466414498935
epoch: 10
epoch: 10 loss: 0.19170140702260974
eval loss: 0.1983

epoch: 0
epoch: 0 loss: 0.30675255294334025
eval loss: 0.2523324185841425 0.33043347320339367
epoch: 1
epoch: 1 loss: 0.24322392428209505
eval loss: 0.22684039908639345 0.2984399328308906
epoch: 2
epoch: 2 loss: 0.23005927614710858
eval loss: 0.2215354950919223 0.29013876566525304
epoch: 3
epoch: 3 loss: 0.21805803409069438
eval loss: 0.2105247092112688 0.2749243725963975
epoch: 4
epoch: 4 loss: 0.21122834668303445
eval loss: 0.2048872286408086 0.268073719707412
epoch: 5
epoch: 5 loss: 0.20428677541255644
eval loss: 0.20161806564219875 0.2641234469247521
epoch: 6
epoch: 6 loss: 0.19843234963777687
eval loss: 0.19669552856487388 0.2569515582608596
epoch: 7
epoch: 7 loss: 0.19304488797458272
eval loss: 0.19212128865824096 0.25119577921496866
epoch: 8
epoch: 8 loss: 0.1911657480170006
eval loss: 0.19257967119580024 0.25309481369730913
epoch: 9
epoch: 9 loss: 0.18706884036313812
eval loss: 0.18655658723625546 0.24505390539759267
epoch: 10
epoch: 10 loss: 0.18278405414736484
eval loss: 0.18

epoch: 0
epoch: 0 loss: 0.3186809025693589
eval loss: 0.25563235724675304 0.3399906603882233
epoch: 1
epoch: 1 loss: 0.24899763956752352
eval loss: 0.23960901906615845 0.3187164446209102
epoch: 2
epoch: 2 loss: 0.23725210784761977
eval loss: 0.22782554517314005 0.3060724300978687
epoch: 3
epoch: 3 loss: 0.2280039469471788
eval loss: 0.22402556164129198 0.29897488948755346
epoch: 4
epoch: 4 loss: 0.2219356172351873
eval loss: 0.21529204190886836 0.28812408690873914
epoch: 5
epoch: 5 loss: 0.21485844938403972
eval loss: 0.20949259841245368 0.2803030939191537
epoch: 6
epoch: 6 loss: 0.20978094146979998
eval loss: 0.20069114980166766 0.26588142780823376
epoch: 7
epoch: 7 loss: 0.20331028027014117
eval loss: 0.2012952400990188 0.26772442443195804
epoch: 8
epoch: 8 loss: 0.19928223644912044
eval loss: 0.19163303600678386 0.25541079577110554
epoch: 9
epoch: 9 loss: 0.19463957058702935
eval loss: 0.19049495265234442 0.25396827632023566
epoch: 10
epoch: 10 loss: 0.19165836872160996
eval loss: 0

epoch: 0
epoch: 0 loss: 0.3157238678187507
eval loss: 0.2576153657947001 0.3377577131036481
epoch: 1
epoch: 1 loss: 0.2510768992136239
eval loss: 0.23742329748919697 0.31270889240037714
epoch: 2
epoch: 2 loss: 0.2354397550483699
eval loss: 0.2237385127435247 0.2941019187947601
epoch: 3
epoch: 3 loss: 0.22595574882957037
eval loss: 0.22098277221921564 0.29030401049185484
epoch: 4
epoch: 4 loss: 0.22111148596798813
eval loss: 0.20982624987443171 0.27757058014932456
epoch: 5
epoch: 5 loss: 0.21669163576967546
eval loss: 0.21468091751586418 0.2835393538647315
epoch: 6
epoch: 6 loss: 0.20829267447481287
eval loss: 0.20231908177986577 0.26724127604386033
epoch: 7
epoch: 7 loss: 0.20387064700707458
eval loss: 0.19776693690683575 0.26190377329539777
epoch: 8
epoch: 8 loss: 0.19937918403364366
eval loss: 0.19228317236222317 0.2563478537488796
epoch: 9
epoch: 9 loss: 0.19474765378688938
eval loss: 0.19136781999897057 0.2533369214336424
epoch: 10
epoch: 10 loss: 0.19101062998470916
eval loss: 0.1

In [10]:
def predict_batch(model, data, device):
    # batch x seq_len x target_size
    with torch.no_grad():
        pred = model(data["sequence"].to(device), data["bpp"].to(device))
        pred = pred.detach().cpu().numpy()
    return_values = []
    ids = data["ids"]
    for idx, p in enumerate(pred):
        id_ = ids[idx]
        assert p.shape == (model.pred_len, len(target_cols))
        for seqpos, val in enumerate(p):
            assert len(val) == len(target_cols)
            dic = {key: val for key, val in zip(target_cols, val)}
            dic["id_seqpos"] = f"{id_}_{seqpos}"
            return_values.append(dic)
    return return_values


def predict_data(model, loader, device, batch_size):
    data_list = []
    for i, data in enumerate(progress_bar(loader)):
        data_list += predict_batch(model, data, device)
    expected_length = model.pred_len * len(loader) * batch_size
    assert len(data_list) == expected_length, f"len = {len(data_list)} expected = {expected_length}"
    return data_list


### Prediction

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
base_test_data = pd.read_json(str(Path(BASE_PATH) / 'test.json'), lines=True)
public_df = base_test_data.query("seq_length == 107").copy()
private_df = base_test_data.query("seq_length == 130").copy()
print(f"public_df: {public_df.shape}")
print(f"private_df: {private_df.shape}")
public_df = public_df.reset_index()
private_df = private_df.reset_index()
pub_loader = create_loader(public_df, BATCH_SIZE, is_test=True)
pri_loader = create_loader(private_df, BATCH_SIZE, is_test=True)
pred_df_list = []
c = 0
for fold in range(5):
    model_load_path = f"./model_prediction/model-{fold}.pt"
    ae_model0 = AEModel()
    ae_model1 = AEModel()
    model_pub = FromAeModel(pred_len=107, seq=ae_model0.seq)
    model_pub = model_pub.to(device)
    model_pri = FromAeModel(pred_len=130, seq=ae_model1.seq)
    model_pri = model_pri.to(device)
    state_dict = torch.load(model_load_path, map_location=device)
    model_pub.load_state_dict(state_dict)
    model_pri.load_state_dict(state_dict)
    del state_dict

    data_list = []
    data_list += predict_data(model_pub, pub_loader, device, BATCH_SIZE)
    data_list += predict_data(model_pri, pri_loader, device, BATCH_SIZE)
    pred_df = pd.DataFrame(data_list, columns=["id_seqpos"] + target_cols)
    print(pred_df.head())
    print(pred_df.tail())
    pred_df_list.append(pred_df)
    c += 1
data_dic = dict(id_seqpos=pred_df_list[0]["id_seqpos"])
for col in target_cols:
    vals = np.zeros(pred_df_list[0][col].shape[0])
    for df in pred_df_list:
        vals += df[col].values
    data_dic[col] = vals / float(c)
pred_df_avg = pd.DataFrame(data_dic, columns=["id_seqpos"] + target_cols)
print(pred_df_avg.head())
pred_df_avg.to_csv("./submission.csv", index=False)


public_df: (629, 7)
private_df: (3005, 7)
(1, 107, 107, 3)
(1, 130, 130, 3)


        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.776515     0.824674  1.813194    0.563424  0.756904
1  id_00073f8be_1    2.519537     3.295120  4.079534    3.278911  2.944239
2  id_00073f8be_2    1.571656     0.658931  0.626493    0.653813  0.735708
3  id_00073f8be_3    1.294815     1.216727  1.190936    1.761870  1.827749
4  id_00073f8be_4    0.906778     0.595592  0.558260    0.938419  0.988071
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.096043     0.392177  0.354100    0.500296   
457949  id_ffda94f24_126    0.377521     0.519269  1.019004    0.642925   
457950  id_ffda94f24_127    0.599662     0.269538  0.383775    0.284465   
457951  id_ffda94f24_128   -0.004982     0.451013  0.354492    0.449578   
457952  id_ffda94f24_129    0.188647     0.457067  0.477021    0.670518   

         deg_50C  
457948  0.313625  
457949  0.836353  
457950  0.458611  
457951  0.294836  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.825423     0.584091  1.777551    0.435887  0.720499
1  id_00073f8be_1    2.367449     3.301724  4.139466    3.217031  2.779341
2  id_00073f8be_2    1.698558     0.455184  0.624785    0.606408  0.659401
3  id_00073f8be_3    1.301391     1.243653  1.344100    1.777979  1.774092
4  id_00073f8be_4    0.788625     0.510652  0.432923    0.903827  0.849623
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.213770     0.457415  0.370763    0.579796   
457949  id_ffda94f24_126    0.498825     0.384016  0.493283    0.573369   
457950  id_ffda94f24_127    0.713420     0.041531  0.087966    0.242617   
457951  id_ffda94f24_128    0.364876     0.053125  0.010299    0.277405   
457952  id_ffda94f24_129    0.598143     0.606256  0.325913    0.772884   

         deg_50C  
457948  0.361806  
457949  0.492423  
457950  0.320162  
457951  0.183381  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.874582     0.698138  2.006648    0.601986  0.738487
1  id_00073f8be_1    2.397415     3.099681  3.949216    3.409038  2.769058
2  id_00073f8be_2    1.704391     0.446882  0.469911    0.619796  0.591133
3  id_00073f8be_3    1.391434     1.255318  1.268791    1.800835  1.835984
4  id_00073f8be_4    0.980037     0.664914  0.454597    0.952523  0.893061
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125   -0.071863     0.328742  0.281375    0.474900   
457949  id_ffda94f24_126    0.201119     0.419974  0.498373    0.467236   
457950  id_ffda94f24_127    0.382366     0.191364  0.234392    0.220138   
457951  id_ffda94f24_128   -0.050018     0.227023  0.227780    0.477891   
457952  id_ffda94f24_129    0.112536     0.327735  0.374343    0.318066   

         deg_50C  
457948  0.232758  
457949  0.589303  
457950  0.287520  
457951  0.188397  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.744365     0.596305  1.811344    0.507739  0.769864
1  id_00073f8be_1    2.421815     2.954833  3.851343    3.061374  2.664587
2  id_00073f8be_2    1.478380     0.728263  0.701230    0.779583  0.640335
3  id_00073f8be_3    1.236575     1.100497  1.078285    1.434915  1.870876
4  id_00073f8be_4    0.781425     0.519107  0.420098    0.813529  0.871711
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.301435     0.312064  0.369389    0.454155   
457949  id_ffda94f24_126    0.428111     0.257571  0.899649    0.438248   
457950  id_ffda94f24_127    0.621575     0.143875  0.155460    0.216259   
457951  id_ffda94f24_128    0.184389     0.151896  0.061671    0.231299   
457952  id_ffda94f24_129    0.107019     0.113484  0.304010    0.206129   

         deg_50C  
457948  0.146175  
457949  0.552483  
457950  0.125820  
457951 -0.163225  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.685752     0.735046  1.918670    0.567634  0.796215
1  id_00073f8be_1    2.284224     3.042243  4.571076    2.981692  2.860991
2  id_00073f8be_2    1.666587     0.742518  0.860221    0.886663  0.970798
3  id_00073f8be_3    1.350224     1.100080  1.309342    1.608225  1.826831
4  id_00073f8be_4    0.709335     0.624428  0.582916    0.789661  0.819878
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.058724     0.341151  0.419136    0.462688   
457949  id_ffda94f24_126    0.181126     0.532955  1.350785    0.663485   
457950  id_ffda94f24_127    0.424188     0.146118  0.377525    0.258806   
457951  id_ffda94f24_128    0.028309     0.028396  0.164784    0.193107   
457952  id_ffda94f24_129    0.125651     0.609985  0.674199    0.667489   

         deg_50C  
457948  0.008354  
457949  0.731701  
457950  0.465794  
457951 -0.009807  
4579